---
title: "Multinomial Logit Model"
author: "Ruoqi Li"
date: today
---


This assignment expores two methods for estimating the MNL model: (1) via Maximum Likelihood, and (2) via a Bayesian approach using a Metropolis-Hastings MCMC algorithm. 


## 1. Likelihood for the Multi-nomial Logit (MNL) Model

Suppose we have $i=1,\ldots,n$ consumers who each select exactly one product $j$ from a set of $J$ products. The outcome variable is the identity of the product chosen $y_i \in \{1, \ldots, J\}$ or equivalently a vector of $J-1$ zeros and $1$ one, where the $1$ indicates the selected product. For example, if the third product was chosen out of 3 products, then either $y=3$ or $y=(0,0,1)$ depending on how we want to represent it. Suppose also that we have a vector of data on each product $x_j$ (eg, brand, price, etc.). 

We model the consumer's decision as the selection of the product that provides the most utility, and we'll specify the utility function as a linear function of the product characteristics:

$$ U_{ij} = x_j'\beta + \epsilon_{ij} $$

where $\epsilon_{ij}$ is an i.i.d. extreme value error term. 

The choice of the i.i.d. extreme value error term leads to a closed-form expression for the probability that consumer $i$ chooses product $j$:

$$ \mathbb{P}_i(j) = \frac{e^{x_j'\beta}}{\sum_{k=1}^Je^{x_k'\beta}} $$

For example, if there are 3 products, the probability that consumer $i$ chooses product 3 is:

$$ \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{e^{x_1'\beta} + e^{x_2'\beta} + e^{x_3'\beta}} $$

A clever way to write the individual likelihood function for consumer $i$ is the product of the $J$ probabilities, each raised to the power of an indicator variable ($\delta_{ij}$) that indicates the chosen product:

$$ L_i(\beta) = \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} = \mathbb{P}_i(1)^{\delta_{i1}} \times \ldots \times \mathbb{P}_i(J)^{\delta_{iJ}}$$

Notice that if the consumer selected product $j=3$, then $\delta_{i3}=1$ while $\delta_{i1}=\delta_{i2}=0$ and the likelihood is:

$$ L_i(\beta) = \mathbb{P}_i(1)^0 \times \mathbb{P}_i(2)^0 \times \mathbb{P}_i(3)^1 = \mathbb{P}_i(3) = \frac{e^{x_3'\beta}}{\sum_{k=1}^3e^{x_k'\beta}} $$

The joint likelihood (across all consumers) is the product of the $n$ individual likelihoods:

$$ L_n(\beta) = \prod_{i=1}^n L_i(\beta) = \prod_{i=1}^n \prod_{j=1}^J \mathbb{P}_i(j)^{\delta_{ij}} $$

And the joint log-likelihood function is:

$$ \ell_n(\beta) = \sum_{i=1}^n \sum_{j=1}^J \delta_{ij} \log(\mathbb{P}_i(j)) $$



## 2. Simulate Conjoint Data

We will simulate data from a conjoint experiment about video content streaming services. We elect to simulate 100 respondents, each completing 10 choice tasks, where they choose from three alternatives per task. For simplicity, there is not a "no choice" option; each simulated respondent must select one of the 3 alternatives. 

Each alternative is a hypothetical streaming offer consistent of three attributes: (1) brand is either Netflix, Amazon Prime, or Hulu; (2) ads can either be part of the experience, or it can be ad-free, and (3) price per month ranges from \$4 to \$32 in increments of \$4.

The part-worths (ie, preference weights or beta parameters) for the attribute levels will be 1.0 for Netflix, 0.5 for Amazon Prime (with 0 for Hulu as the reference brand); -0.8 for included adverstisements (0 for ad-free); and -0.1*price so that utility to consumer $i$ for hypothethical streaming service $j$ is 

$$
u_{ij} = (1 \times Netflix_j) + (0.5 \times Prime_j) + (-0.8*Ads_j) - 0.1\times Price_j + \varepsilon_{ij}
$$

where the variables are binary indicators and $\varepsilon$ is Type 1 Extreme Value (ie, Gumble) distributed.

The following code provides the simulation of the conjoint data.

:::: {.callout-note collapse="true"}

In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

n_respondents = 100
n_tasks = 10
n_alternatives = 3

brands = ['Hulu', 'Netflix', 'Prime']
prices = np.arange(4, 36, 4)  # $4 to $32
ads = [0, 1]  # 0 = no ads, 1 = has ads

# Utility coefficients
beta = {
    'Netflix': 1.0,
    'Prime': 0.5,
    'Ads': -0.8,
    'Price': -0.1
}

rows = []

for resp in range(n_respondents):
    for task in range(n_tasks):
        alts = []
        for alt in range(n_alternatives):
            brand = np.random.choice(brands)
            ad = np.random.choice(ads)
            price = np.random.choice(prices)
            
            # Indicator vars
            netflix = 1 if brand == 'Netflix' else 0
            prime = 1 if brand == 'Prime' else 0
            
            # Utility calculation
            utility = (beta['Netflix'] * netflix +
                       beta['Prime'] * prime +
                       beta['Ads'] * ad +
                       beta['Price'] * price +
                       np.random.gumbel())
            
            alts.append((brand, ad, price, utility))
        
        # Select the alternative with highest utility
        chosen = np.argmax([u for (_, _, _, u) in alts])
        
        for idx, (brand, ad, price, utility) in enumerate(alts):
            rows.append({
                'Respondent': resp,
                'Task': task,
                'Alternative': idx,
                'Chosen': 1 if idx == chosen else 0,
                'Brand': brand,
                'Netflix': 1 if brand == 'Netflix' else 0,
                'Prime': 1 if brand == 'Prime' else 0,
                'Ads': ad,
                'Price': price
            })

df = pd.DataFrame(rows)

::::



## 3. Preparing the Data for Estimation

The "hard part" of the MNL likelihood function is organizing the data, as we need to keep track of 3 dimensions (consumer $i$, covariate $k$, and product $j$) instead of the typical 2 dimensions for cross-sectional regression models (consumer $i$ and covariate $k$). The fact that each task for each respondent has the same number of alternatives (3) helps.  In addition, we need to convert the categorical variables for brand and ads into binary variables.

- Converting the categorical brand variable into two binary indicators: `Netflix`, `Prime` (with Hulu as the reference category),
- Including indicators for ads (0 = ad-free, 1 = with ads),
- Including the monthly price,
- Tracking which alternative was chosen by the respondent.

In [ ]:
df['Group'] = df['Respondent'] * n_tasks + df['Task']
df_prepared = df[['Respondent', 'Task', 'Alternative', 'Group', 'Chosen',
                  'Netflix', 'Prime', 'Ads', 'Price']]

## 4. Estimation via Maximum Likelihood

In [ ]:
#| echo: false
from scipy.optimize import minimize
from scipy.special import logsumexp

# Features and response
X = df_prepared[['Netflix', 'Prime', 'Ads', 'Price']].values
y = df_prepared['Chosen'].values
groups = df_prepared['Group'].values

# Unique groups for choice sets
unique_groups = np.unique(groups)

To estimate the parameters of the MNL model, we first implement the log-likelihood function. Each individual chooses one alternative from a set of three, and the probability of choosing an alternative is modeled using the softmax function derived from the utility specification.

The negative log-likelihood function is computed as follows:

In [ ]:
# Log-likelihood function
def neg_log_likelihood(beta):
    utilities = X @ beta  # linear part
    log_likelihood = 0
    for group in unique_groups:
        mask = groups == group
        u = utilities[mask]
        y_group = y[mask]
        log_prob = u - logsumexp(u)  # log softmax
        log_likelihood += np.sum(y_group * log_prob)
    return -log_likelihood  # negative for minimization

After defining the log-likelihood function, we use scipy.optimize.minimize() to estimate the model parameters. We apply the BFGS algorithm to find the maximum likelihood estimates (MLEs), and use the inverse Hessian matrix to compute standard errors and 95% confidence intervals for each parameter.

In [ ]:
# Initial guess
beta_init = np.zeros(X.shape[1])

# Estimate using scipy.optimize
result = minimize(neg_log_likelihood, beta_init, method='BFGS')
beta_hat = result.x
hessian_inv = result.hess_inv

# Compute standard errors from Hessian
se = np.sqrt(np.diag(hessian_inv))

# 95% CI
z = 1.96
conf_int = np.vstack((beta_hat - z * se, beta_hat + z * se)).T

# Results table
params = ['beta_Netflix', 'beta_Prime', 'beta_Ads', 'beta_Price']
results_df = pd.DataFrame({
    'Parameter': params,
    'Estimate': beta_hat,
    'Std. Error': se,
    'CI Lower (95%)': conf_int[:, 0],
    'CI Upper (95%)': conf_int[:, 1]
})
results_df

### The interpretation of each coefficient: 

- **`beta_Netflix = 1.04`**: Respondents strongly prefer Netflix over the baseline category (Hulu). The 95% confidence interval \([0.99, 1.09]\) does not include zero, confirming statistical significance.
- **`beta_Prime = 0.58`**: Amazon Prime is also preferred relative to Hulu, though to a lesser extent than Netflix. The effect is statistically significant.
- **`beta_Ads = -0.81`**: There is a strong negative impact of advertisements on utility, as indicated by the large negative estimate and a confidence interval well below zero.
- **`beta_Price = -0.097`**: Price has a small but statistically significant negative effect on utility, consistent with theoretical expectations.



## 5. Estimation via Bayesian Methods

In [ ]:
import numpy as np
def log_prior(beta):
    logp = 0
    logp += -0.5 * np.sum((beta[:3] / 5) ** 2) - 3 * np.log(5 * np.sqrt(2 * np.pi))
    logp += -0.5 * (beta[3] ** 2) - np.log(np.sqrt(2 * np.pi))
    return logp

# Posterior = log-likelihood + log-prior
def log_posterior(beta):
    return -neg_log_likelihood(beta) + log_prior(beta)

n_steps = 11000
burn_in = 1000
dim = 4
beta_current = np.zeros(dim)
log_post_current = log_posterior(beta_current)
samples = np.zeros((n_steps, dim))

# Proposal SDs
proposal_sds = np.array([0.05, 0.05, 0.05, 0.005])
np.random.seed(42)

for step in range(n_steps):
    proposal = beta_current + np.random.normal(0, proposal_sds)
    log_post_proposal = log_posterior(proposal)
    accept_prob = np.exp(log_post_proposal - log_post_current)
    if np.random.rand() < accept_prob:
        beta_current = proposal
        log_post_current = log_post_proposal
    samples[step] = beta_current

posterior_samples = samples[burn_in:]
posterior_samples

| Column Index | Parameter Name  | Interpretation |
|--------------|------------------|----------------|
| `[ , 0]`      | `beta_Netflix`   | User preference for **Netflix**, relative to the reference category (Hulu) |
| `[ , 1]`      | `beta_Prime`     | User preference for **Amazon Prime** |
| `[ , 2]`      | `beta_Ads`       | **Negative impact of ads** (a more negative value indicates greater dislike for ads) |
| `[ , 3]`      | `beta_Price`     | **Negative impact of price** (a more negative value indicates greater sensitivity to price) |

In [ ]:
#| echo: false
import matplotlib.pyplot as plt

# Extract samples for beta_Netflix
beta_netflix_samples = posterior_samples[:, 0]

# Plot trace plot and histogram
fig, ax = plt.subplots(2, 1, figsize=(8, 6), sharex=False)

# Trace plot
ax[0].plot(beta_netflix_samples, color='black', linewidth=0.5)
ax[0].set_title("Trace Plot for beta_Netflix")
ax[0].set_ylabel("Sampled Value")

# Histogram
ax[1].hist(beta_netflix_samples, bins=50, color='skyblue', edgecolor='black', density=True)
ax[1].set_title("Posterior Distribution of beta_Netflix")
ax[1].set_xlabel("beta_Netflix")
ax[1].set_ylabel("Density")

plt.tight_layout()
plt.show()

- **Trace Plot** (Top):  
  The trace plot shows rapid mixing and no obvious trend or drift, indicating that the sampler has converged and is exploring the posterior effectively.

- **Posterior Histogram** (Bottom):  
  The histogram approximates a smooth, bell-shaped distribution centered around the posterior mean. This reflects a stable estimate with symmetric uncertainty.

In [ ]:
#| echo: false
# Compute posterior summary statistics
param_names = ['beta_Netflix', 'beta_Prime', 'beta_Ads', 'beta_Price']
posterior_means = np.mean(posterior_samples, axis=0)
posterior_sds = np.std(posterior_samples, axis=0)
cred_ints = np.percentile(posterior_samples, [2.5, 97.5], axis=0).T

# Combine into a DataFrame
posterior_summary = pd.DataFrame({
    'Parameter': param_names,
    'Posterior Mean': posterior_means,
    'Posterior Std. Dev': posterior_sds,
    '95% CI Lower': cred_ints[:, 0],
    '95% CI Upper': cred_ints[:, 1]
})

posterior_summary

### Interpretation
- `beta_Netflix` and `beta_Prime` are positive, showing that people prefer Netflix and Prime over the base brand (Hulu).
- `beta_Ads` is negative, meaning people dislike ads.
- `beta_Price` is also negative, meaning higher prices reduce utility.
- All 95% credible intervals do **not include 0**, so all four parameters are statistically meaningful. This confirms that the Bayesian estimation gives results consistent with what we expect based on the data.

## 6. Discussion
Suppose we did **not** simulate the data and instead received these results from a real-world conjoint study. Here's what we can conclude:

- `β_Netflix > β_Prime` means that, on average, respondents **prefer Netflix over Amazon Prime**, all else being equal.
- The **positive values** for both brand coefficients (compared to the baseline Hulu) suggest that both Netflix and Prime are generally preferred brands.
- `β_price` is **negative**, which makes intuitive sense — higher prices reduce the likelihood of choosing an alternative. This is consistent with basic economic theory and consumer behavior.
- Overall, the signs and magnitudes of the parameters are reasonable and aligned with real-world expectations, even without knowing the data were simulated.



### Why move beyond the basic MNL model?

The standard Multinomial Logit (MNL) model assumes that **everyone shares the same preferences**. For example:

> Everyone is equally likely to prefer Netflix over Hulu,  
> and equally sensitive to price changes.

But in the real world, this is rarely true:

- Some people **love Netflix**, others don't care.
- Some people are **very price-sensitive**, others are not.
- Some tolerate ads, others strongly avoid them.

To reflect this real-world variation, we need a model that allows **individual-level differences** in preferences. This leads us to the **multi-level (or hierarchical)** model.

Instead of estimating just one set of parameters for the entire population, we assume each person \( i \) has their own preference vector \( \beta_i \), drawn from a population distribution:
\[
\beta_i \sim \mathcal{N}(\mu, \Sigma)
\]

This model captures **both overall trends** and **individual differences**, making it much more realistic for analyzing real-world conjoint data.

### Key Modifications: 
1. **Simulating hierarchical data**:
   - Instead of one global β vector, we assume each respondent \( i \) has their own preference vector \( \beta_i \).
   - These individual \( \beta_i \)s are drawn from a population distribution:
     \[
     \beta_i \sim \mathcal{N}(\mu, \Sigma)
     \]
   - You first draw \( \beta_i \) for each respondent, and then simulate choices using that individual's preferences.

2. **Estimating the model**:
   - You no longer estimate a single \( \beta \), but instead estimate:
     - \( \mu \): the mean of the population distribution,
     - \( \Sigma \): the covariance matrix (individual-level variation),
     - \( \beta_i \): each respondent’s specific preferences (if using full Bayesian methods).
   - This often requires **Bayesian methods** (e.g., MCMC, HMC in PyMC or Stan) or **frequentist approaches** like simulated maximum likelihood.
